In [1]:
import pandas
import json
from pprint import pprint
from connect_sql_db import build_engine
from IPython.display import clear_output

In [2]:
engine = build_engine(database_name="database1",host="35.225.193.21")

# Building Business DataFrame

In [ ]:
dataset = []
for line in open("../../../data/yelp_dataset/yelp_academic_dataset_business.json"):
    dataset.append(json.loads(line))

In [ ]:
for dictionary in dataset:
    dictionary["categories"] = 0
    dictionary["attributes"] = 0
    dictionary["hours"] = 0

In [ ]:
business_df = pandas.DataFrame(dataset)

In [ ]:
business_df.drop(
    [
    "categories",
    "attributes",
    "hours"
    ],
    axis = 1,
    inplace = True
)

In [ ]:
business_df.count()

# Building Attributes DataFrame

In [ ]:
dataset = []
for line in open("../../../data/yelp_dataset/yelp_academic_dataset_business.json"):
    dataset.append(json.loads(line))

In [ ]:
unique_categories = []
for i in dataset:
    if i["categories"] != None:
        for v in i["categories"].split(","):
            unique_categories.append((i["business_id"],v.strip().replace(" ","_").replace("/","_").replace("\\","_").replace("&","and").lower()))
    elif i["categories"] == None:
        unique_categories.append((i["business_id"],"None"))
        
unique_categories = list(set(unique_categories))
pprint(unique_categories[0:10])

In [ ]:
business_df.head()

In [ ]:
attributes_df = business_df[["business_id"]].copy()

In [ ]:
columns = list(set([i[1]for i in unique_categories]))
counter=1
for i in columns:
    print(f"{counter/len(columns)}% complete {counter} of {len(columns)} columns created name: {i}")
    attributes_df[i] = 0
    counter+=1

In [ ]:
counter=1
clear_me = 0
for i in unique_categories:
    print(f"{counter/len(unique_categories)}% complete: cells affected {counter} of {len(unique_categories)}")
    attributes_df.loc[attributes_df.business_id == i[0],i[1]] = 1
    counter+=1
    clear_me += 1
    if clear_me == 50 and counter != len(unique_categories):
        clear_output()
        clear_me = 0

In [ ]:
attributes_df.head()

In [ ]:
attributes_df.count()

In [ ]:
attributes_df.to_csv("../../../data/attributes_table.csv",index=False)

# Attributes DataFrame pt. 2

In [ ]:
dataset = []
for line in open("../../../data/yelp_dataset/yelp_academic_dataset_business.json"):
    dataset.append(json.loads(line))

In [ ]:
attributes_df = pandas.read_csv("../../../data/attributes.csv")

In [ ]:
unique_attributes = []
for i in dataset:
    if i["attributes"] != None:
        for v in dict(i["attributes"]).keys():
            unique_attributes.append(v)

unique_attributes = list(set(unique_attributes))
unique_attributes

In [ ]:
for i in unique_attributes:
    attributes_df[i] = 0

In [ ]:
bool_list = []
for i in dataset:
    if i["attributes"] != None:
        for v,x in dict(i["attributes"]).items():
            if "{" in x or "}" in x:
                bool_list.append((i["business_id"],v,"dict_col"))
            else:
                bool_list.append((i["business_id"],v,x.strip().replace("u'","").replace("'","").replace(" ","_").replace("/","_").replace("\\","_").replace("&","and").lower()))
    elif i["attributes"] == None:
        bool_list.append((i["business_id"],v,"None"))
    
bool_list[:10]   

In [ ]:
counter = 1
clear_me = 0
for i,v,x in bool_list:
    print(f"{counter/len(bool_list)}% complete: cell {counter} of {len(bool_list)}")
    attributes_df.loc[attributes_df.business_id == i, v] = x
    counter+=1
    clear_me += 1
    if clear_me == 50 and counter != len(bool_list):
        clear_output()
        clear_me = 0
    

In [ ]:
attributes_df.to_csv("../../../data/attributes_table_pt2.csv",index=False)

In [ ]:
attributes_df.count()

In [ ]:
business_df.count()

In [ ]:
final_table = pandas.merge(business_df,attributes_df,how="left",on="business_id")

In [ ]:
final_table.to_csv("~/Desktop/merged_table_1.csv", index = False)

In [ ]:
final_table.columns = [i.lower().replace(" ","_").replace("&","and").replace("(","").replace(")","") for i in final_table.columns]


In [ ]:
filtered_df.to_sql(
    "merged_table_1",
    con = engine,
    index = False, 
    method = 'multi',
    chunksize = 10000,
    if_exists="replace"
)

In [ ]:
len(final_table.columns)

In [3]:
final_table = pandas.read_csv("~/Desktop/merged_table.csv",low_memory = False)

In [4]:
len(final_table)

209393

In [5]:
food_lst=[]
for i in final_table.columns:
    if "food" in i or "restaurant" in i:
        food_lst.append(i)
len(food_lst)

31

In [6]:
food_lst

['pop-up_restaurants',
 'specialty_food',
 'ethnic_food',
 'swiss_food',
 'food_court',
 'food_trucks',
 'food_safety_training',
 'live_raw_food',
 'food_delivery_services',
 'do-it-yourself_food',
 'fast_food',
 'restaurants',
 'soul_food',
 'comfort_food',
 'imported_food',
 'food_tours',
 'seafood_markets',
 'food_stands',
 'food_banks',
 'restaurant_supplies',
 'pub_food',
 'seafood',
 'food',
 'restaurantsattire',
 'restaurantsdelivery',
 'restaurantsreservations',
 'restaurantstableservice',
 'restaurantscounterservice',
 'restaurantstakeout',
 'restaurantspricerange2',
 'restaurantsgoodforgroups']

In [45]:
filtered_df = final_table.loc[
    (final_table[food_lst[0]] == 1) |
    (final_table[food_lst[1]] == 1) |
    (final_table[food_lst[2]] == 1) |
    (final_table[food_lst[3]] == 1) |
    (final_table[food_lst[4]] == 1) |
    (final_table[food_lst[5]] == 1) |
    (final_table[food_lst[6]] == 1) |
    (final_table[food_lst[7]] == 1) |
    (final_table[food_lst[8]] == 1) |
    (final_table[food_lst[9]] == 1) |
    (final_table[food_lst[10]] == 1) |
#     (final_table[food_lst[11]] == 1) |
    (final_table[food_lst[12]] == 1) |
    (final_table[food_lst[13]] == 1) |
    (final_table[food_lst[14]] == 1) |
    (final_table[food_lst[15]] == 1) |
    (final_table[food_lst[16]] == 1) |
    (final_table[food_lst[17]] == 1) |
    (final_table[food_lst[18]] == 1) |
    (final_table[food_lst[19]] == 1) |
    (final_table[food_lst[20]] == 1) |
    (final_table[food_lst[21]] == 1) |
    (final_table[food_lst[22]] == 1) |
    (final_table[food_lst[23]] == 1) |
    (final_table[food_lst[24]] == 1) |
    (final_table[food_lst[25]] == 1) |
    (final_table[food_lst[26]] == 1) |
    (final_table[food_lst[27]] == 1) |
    (final_table[food_lst[28]] == 1) |
    (final_table[food_lst[29]] == 1) |
    (final_table[food_lst[30]] == 1) 
    
].copy()

In [46]:
for i in filtered_df.columns:
    if filtered_df[i].dtypes == int:
        if sum(filtered_df[i].tolist()) == 0:
            filtered_df.drop(i, axis = 1, inplace=True)

In [47]:
filtered_df.isna().sum()[filtered_df.isna().sum() > 0 ]

address        847
postal_code    154
dtype: int64

In [10]:
filtered_df.business_id.count()

80505

In [11]:
len(filtered_df.columns)

974

In [12]:
len(final_table.columns)

1387

In [13]:
filtered_df.to_csv("~/Desktop/merged_table_1.csv",index=False)

In [22]:
filtered_df.drop("business_id",axis=1).isna().sum()[filtered_df.isna().sum()> 0]

address        912
postal_code    183
dtype: int64

In [25]:
for i in filtered_df.columns:
    if filtered_df[i].dtypes == int:
        if filtered_df[i].sum() < 50:
            print(i)

adult_entertainment
sports_clubs
haitian
radio_stations
musicians
utilities
kiosk
formal_wear
resorts
empanadas
internet_service_providers
leisure_centers
pediatric_dentists
furniture_reupholstery
vocational_and_technical_school
horse_racing
knitting_supplies
mailbox_centers
medical_spas
home_organization
childbirth_education
bike_repair_maintenance
pet_boarding
video_game_stores
videos_and_video_game_rental
pop-up_restaurants
day_camps
couriers_and_delivery_services
session_photography
windows_installation
boat_repair
police_departments
architectural_tours
cultural_center
atv_rentals_tours
general_dentistry
rock_climbing
sugar_shacks
backshop
team_building_activities
escape_games
mini_golf
hot_tub_and_pool
beer_tours
ukrainian
vacation_rentals
hungarian
skydiving
oil_change_stations
estate_liquidation
marketing
senegalese
comedy_clubs
shipping_centers
tattoo_removal
pancakes
lakes
nurseries_and_gardening
party_equipment_rentals
blow_dry_out_services
window_washing
walking_tours
gyms
t

In [43]:
filtered_df.loc[filtered_df.lingerie > 0][food_lst].values

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, '0', '0', '0', '0', '0', '0', '0', '0']], dtype=object)

In [44]:
food_lst[11]

'restaurants'